In [73]:
import pandas as pd
import requests
import json
from requests.auth import HTTPBasicAuth

In [74]:
week = 10
year = 2024
filepath = "market_prices_current.csv"

In [75]:
def request_data():
    query = {'source': 'Pinnacle'}
    response = requests.get(
              'https://api.ubiqanalytics.com/eventPrice?source=PS3838&sport=Football&eventStartTimeFrom=2024-10-01',
              auth=HTTPBasicAuth('quant', 'FastBeefProfit3'),params=query
    )
    return response

def parse_data(response):
    json_data = json.loads(response.text)
    matches = {}
    for i in json_data:
        if i['event']['competition']['name'] == 'NFL':
            matches[i['eventId']] = {'away_team' : i['event']['participants'][0]['name'],'away_team_id' : i['event']['participants'][0]['id'],  'home_team': i['event']['participants'][1]['name'],'home_team_id': i['event']['participants'][1]['id']}
            for j in i['event']['selections']:
                if j['key']['type'] == 'AsianHandicap' and j['key']['outcomes'][0]['outcomePeriod'] == 'Full' and j['priority'] == 1:
                    if j['key']['outcomes'][0]['participantId'] == matches[i['eventId']]['away_team_id']:
                        matches[i['eventId']]['away_market'] = {'spread' : {'line':j['key']['outcomes'][0]['line'],'price':j['priceLimit']['price']['value']}}
                    if j['key']['outcomes'][0]['participantId'] == matches[i['eventId']]['home_team_id']:
                        matches[i['eventId']]['home_market'] = {'spread' : {'line':j['key']['outcomes'][0]['line'],'price':j['priceLimit']['price']['value']}}
                if j['key']['type'] == 'OverUnder' and j['key']['outcomes'][0]['outcomePeriod'] == 'Full' and j['priority'] == 1 and j['key']['unit'] == 'Point':
                    if j['key']['outcomes'][0]['outcomeType'] == 'Under':
                        matches[i['eventId']]['total_market_under'] = {'line':j['key']['outcomes'][0]['line'],'price':j['priceLimit']['price']['value']}
                    else:
                        matches[i['eventId']]['total_market_over'] = {'line':j['key']['outcomes'][0]['line'],'price':j['priceLimit']['price']['value']}
    all_data = []
    for i in matches.keys():
        row_data = [i,matches[i]['away_team'],matches[i]['home_team']]
        try:
            if abs(matches[i]['away_market']['spread']['line']) == abs(matches[i]['home_market']['spread']['line']):
                row_data = row_data + [matches[i]['away_market']['spread']['price'],matches[i]['away_market']['spread']['line'],matches[i]['home_market']['spread']['price'],matches[i]['home_market']['spread']['line']]
        except:
            row_data = row_data + ['NaN','NaN','NaN','NaN']
        try:
            if matches[i]['total_market_under']['line'] == matches[i]['total_market_over']['line']:
                row_data = row_data + [matches[i]['total_market_under']['line'],matches[i]['total_market_under']['price'],matches[i]['total_market_over']['price']]
        except:
            row_data = row_data + ['NaN','NaN','NaN']
        all_data.append(row_data)
    df = pd.DataFrame(columns = ['id','away team','home team','away spread price', 'away spread', 'home spread price', 'home spread','total line', 'under price', 'over price'],data= all_data)

    return df


In [76]:
df = parse_data(request_data())

In [77]:
nameMap = {'Arizona Cardinals': 'ARI',
 'Atlanta Falcons': 'ATL',
 'Baltimore Ravens': 'BAL',
 'Buffalo Bills': 'BUF',
 'Carolina Panthers': 'CAR',
 'Chicago Bears': 'CHI',
 'Cincinnati Bengals': 'CIN',
 'Cleveland Browns': 'CLE',
 'Dallas Cowboys': 'DAL',
 'Denver Broncos': 'DEN',
 'Detroit Lions': 'DET',
 'Green Bay Packers': 'GB',
 'Houston Texans': 'HOU',
 'Indianapolis Colts': 'IND',
 'Jacksonville Jaguars': 'JAX',
 'Kansas City Chiefs': 'KC',
 'Las Vegas Raiders': 'OAK',
 'Los Angeles Chargers': 'LAC',
 'Los Angeles Rams': 'LAR',
 'Miami Dolphins': 'MIA',
 'Minnesota Vikings': 'MIN',
 'New England Patriots': 'NE',
 'New Orleans Saints': 'NO',
 'New York Giants': 'NYG',
 'New York Jets': 'NYJ',
 'Philadelphia Eagles': 'PHI',
 'Pittsburgh Steelers': 'PIT',
 'San Francisco 49ers': 'SF',
 'Seattle Seahawks': 'SEA',
 'Tampa Bay Buccaneers': 'TB',
 'Tennessee Titans': 'TEN',
 'Washington Commanders': 'WAS'}

In [78]:
df["away team"] = df["away team"].map(nameMap)
df["home team"] = df["home team"].map(nameMap)

In [79]:
df["map"] = df.apply(lambda x: "{}_{:02d}_{}_{}".format(year, week, x["away team"], x["home team"]), axis=1)


In [80]:
df.to_csv(filepath)

In [81]:
df

,id,away team,home team,away spread price,away spread,home spread price,home spread,total line,under price,over price,map
0,125878,IND,MIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024_10_IND_MIN
1,125879,TB,KC,-105.0,8.5,-105.0,-8.5,45.5,-111.0,-102.0,2024_10_TB_KC
2,126742,CIN,BAL,-124.0,6.5,110.0,-6.5,52.5,-103.0,-111.0,2024_10_CIN_BAL
3,126743,NYG,CAR,-107.0,-4.5,-105.0,4.5,41.5,-108.0,-106.0,2024_10_NYG_CAR
4,126745,MIN,JAX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024_10_MIN_JAX
5,126749,SF,TB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024_10_SF_TB
6,126744,BUF,IND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024_10_BUF_IND
7,126746,PIT,WAS,-108.0,2.5,-104.0,-2.5,44.5,-108.0,-106.0,2024_10_PIT_WAS
8,126747,ATL,NO,-124.0,-3.0,110.0,3.0,48.0,-115.0,101.0,2024_10_ATL_NO
9,126748,DEN,KC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024_10_DEN_KC


In [19]:
request_data()

<Response [500]>